In [8]:
# import numpy as np
# import cv2
# import matplotlib.pyplot as plt
# %matplotlib qt

#importing some useful packages
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
from enum import Enum
from collections import namedtuple
import pickle
%matplotlib inline

# %matplotlib qt


In [2]:
import os

class File:
    @staticmethod
    def list_folder(folder_path):
        return os.listdir(folder_path)

In [3]:
class ImageViewer:
    @staticmethod
    def display_image(cv_image):        
        result = plt.imshow(cv_image.to_RGB().image_data)
        
        plt.colorbar()
        plt.tight_layout()
        plt.show()
        
        return result





In [4]:
class LoadAs(Enum):
    Grayscale = 1
    ColorBGR = 2
    ColorRGB = 3
    ColorHSV = 4
    ColorHSL = 5
    
class CvImage:
    def __init__(self):
        self.load_as = None
        self.image_data = np.array([], dtype=np.int32)

    @staticmethod
    def create_black_image(width, height, channel_count = 3):
        image_data = np.zeros((height, width, channel_count), np.uint8)

        return CvImage.from_cv_image_data(image_data, LoadAs.ColorBGR)

    @staticmethod
    def load_from_file(filepath, load_as = LoadAs.ColorBGR):
        instance = CvImage()
        instance.load_as = load_as
        if load_as == LoadAs.Grayscale:
            instance.image_data = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        else:
            instance.image_data = cv2.imread(filepath)
            
        return instance
    
    @staticmethod
    def from_cv_image_data(cv_image_data, load_as = LoadAs.ColorBGR):
        
#         if cv_image_data.shape[0] == 0:
#             return None
        
        instance = CvImage()
        instance.load_as = load_as
        instance.image_data = cv_image_data
        
        return instance
    
    @staticmethod
    def load_from_image_data(cv_image_data, load_as = LoadAs.ColorBGR):
        return CvImage.from_cv_image_data(cv_image_data, load_as)

    def _get_intensity_m(self, brightness_value):
        intensitym = np.ones(self.image_data.shape, dtype="uint8") * brightness_value
        
        return intensitym
    
    def darken(self, darken_value):
        intensitym = self._get_intensity_m(darken_value)
        darkened_image_data = cv2.subtract(self.image_data, intensitym)
        
        return CvImage.load_from_image_data(darkened_image_data, self.load_as)
    
    def brighten(self, darken_value):
        intensitym = self._get_intensity_m(darken_value)
        darkened_image_data = cv2.add(self.image_data, intensitym)
        
        return CvImage.load_from_image_data(darkened_image_data, self.load_as)

    def height(self):
#         if self.image_data.shape[0] == 0:
#             return 0
        height, width = self.image_data.shape[:2]
        
        return height
    
    def width(self):
#         if self.image_data.shape[0] == 0:
#             return 0
        height, width = self.image_data.shape[:2]
        
        return width
        

    def channel_count(self):
#         if self.image_data.shape[0] == 0:
#             return 0
        return self.image_data.shape[2]
    
    def to_grayscale(self):
#         if self.image_data.shape[0] == 0:
#             return None        
        grayscale_image = cv2.cvtColor(self.image_data, cv2.COLOR_BGR2GRAY)
        
        return CvImage.from_cv_image_data(grayscale_image, LoadAs.Grayscale)
    

    def mask_and(self, other_image):
        result = cv2.bitwise_and(self.image_data, other_image.image_data)
        
        return CvImage.load_from_image_data(result, self.load_as)
    
    def mask_or(self, other_image):
        result = cv2.bitwise_or(self.image_data, other_image.image_data)
        
        return CvImage.load_from_image_data(result, self.load_as)

    def mask_xor(self, other_image):
        result = cv2.bitwise_xor(self.image_data, other_image.image_data)
        
        return CvImage.load_from_image_data(result, self.load_as)

    def mask_not(self):
        result = cv2.bitwise_not(self.image_data)
        
        return CvImage.load_from_image_data(result, self.load_as)
    
    def to_HSV(self):
        hsv = cv2.cvtColor(self.image_data, cv2.COLOR_RGB2HSV)
        
        return CvImage.load_from_image_data(hsv, LoadAs.ColorHSV)

    def to_HSL(self):
        hsv = cv2.cvtColor(self.image_data, cv2.COLOR_RGB2HLS)
        
        return CvImage.load_from_image_data(hsv, LoadAs.ColorHSL)
    
    def to_RGB(self):
        if self.load_as == LoadAs.Grayscale:
            rgb_image = cv2.cvtColor(self.image_data, cv2.COLOR_GRAY2RGB)
            
            return CvImage.load_from_image_data(rgb_image, LoadAs.ColorRGB)
        else:
            rgb_image = cv2.cvtColor(self.image_data, cv2.COLOR_BGR2RGB)

            return CvImage.load_from_image_data(rgb_image, LoadAs.ColorRGB)
    
    def gaussian_blur(self, kernel_size = 5):
        blurred = cv2.GaussianBlur(self.image_data, (kernel_size, kernel_size), 0)
        
        return CvImage.load_from_image_data(blurred, self.load_as);
    
    def threshold(self, black_threshold, other_color, ttype):
        ret, result = cv2.threshold(self.image_data, black_threshold, other_color, ttype)
        
        return CvImage.load_from_image_data(result, self.load_as)
    
    def threshold_binary(self, black_threshold, other_color):
        return self.threshold(black_threshold, other_color, cv2.THRESH_BINARY)
        
    def threshold_binary_inverse(self, black_threshold, other_color):
        return self.threshold(black_threshold, other_color, cv2.THRESH_BINARY_INV)

    def threshold_truncate(self, black_threshold, other_color):
        return self.threshold(black_threshold, other_color, cv2.THRESH_TRUNC)
    
    def threshold_to_zero(self, black_threshold, other_color):
        return self.threshold(black_threshold, other_color, cv2.THRESH_TOZERO)
    
    def canny(self, low_threshold = 50, high_threshold = 150, aperture_size = 3):
        cannied = cv2.Canny(self.image_data, low_threshold, high_threshold, aperture_size)
        
        return CvImage.load_from_image_data(cannied, self.load_as)    
        
                
    
    def filter_inrange(self, lower_value, upper_value):
        filtered = cv2.inRange(self.image_data, lower_value, upper_value)
        
        return CvImage.load_from_image_data(filtered, self.load_as)
    
    def find_chessboard_corners(self, corners_x, corners_y):
        ret, corners = cv2.findChessboardCorners(self.image_data, (corners_x, corners_y), None)
        
        if ret == True:
            return corners
        else:
            return []
    
    def draw_chessboard_corners(self, corners_x, corners_y):
        gray = self.to_grayscale()
        corners = gray.find_chessboard_corners(corners_x, corners_y)

        # If found, draw corners
        if len(corners) > 0:
            # Draw and display the corners
            copied_image = np.copy(self.image_data)
            cv2.drawChessboardCorners(copied_image, (corners_x, corners_y), corners, True)
        else:
            print("Failed top find chessboard corners")
            return self
        
        return CvImage.load_from_image_data(copied_image, self.load_as)
    
    def undistort(self, camera_calibration_result):
        undistorted = cv2.undistort(self.image_data, camera_calibration_result.calibration_matrix, camera_calibration_result.calibration_distance, None, camera_calibration_result.new_camera_matrix)
        
        return CvImage.load_from_image_data(undistorted, self.load_as)
            
    def unwarp(self, src, dst):
        h,w = self.image_data.shape[:2]
        M = cv2.getPerspectiveTransform(src, dst)
        Minv = cv2.getPerspectiveTransform(dst, src)
        warped = cv2.warpPerspective(img, M, (w,h), flags=cv2.INTER_LINEAR)
        
        return CvImage.load_from_image_data(warped)
#         return warped, M, Minv
        
        

In [5]:
CameraCalibrationResult = namedtuple("CameraCalibrationResult", "calibration_matrix calibration_distance new_camera_matrix")

class Camera:
    def __init__(self):
        self.calibration_matrix = None
        self.calibration_distance = -1
        self.new_camera_matrix = None
        
    def find_correct_corners(self, calibration_image_files):
        index = 0
        corners_hash = {}
        for calibration_image_file in calibration_image_files:
            cvimage = CvImage.load_from_file(calibration_image_file)
            gray = cvimage.to_grayscale()
            corners_found = False
            for i in range(10, 2, -1):
                for j in range(7, 2, -1):
                    print('searching with x:' + str(i) + ', y:' + str(j) )
                    corners = gray.find_chessboard_corners(i, j)
                    if len(corners) > 0:
                        corners_hash[index] = (i, j)
                        corners_found = True
                        break
                if corners_found: 
                    break                
            index += 1
            
        return corners_hash
        
    def calibrate(self, calibration_image_files, points_x, points_y):        
        objpoints = []
        imgpoints = []
        
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

        
        index = 0
        for calibration_image_file in calibration_image_files:            
#             points_x = 8 #corners_hash[index][0]
#             points_y = 6 #corners_hash[index][1]

            objp = np.zeros((points_x * points_y, 3), np.float32)
            objp[:,:2] = np.mgrid[0:points_x,0:points_y].T.reshape(-1, 2)
            
            calibration_image = CvImage.load_from_file(calibration_image_file, LoadAs.Grayscale)
            
            chessboard_corners = calibration_image.find_chessboard_corners(points_x, points_y)
            if len(chessboard_corners) > 0:
                corners2 = cv2.cornerSubPix(calibration_image.image_data,chessboard_corners,(11,11),(-1,-1),criteria)
                                
                imgpoints.append(corners2)
                objpoints.append(objp)
            else:
                print('Not found for index:' + str(index))
                
            index += 1
            
            
        if self.calibration_distance == -1:      
            calibration_image = CvImage.load_from_file(calibration_image_file)
            image_dimension = (img.shape[1], img.shape[0])
            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, image_dimension, None, None)
            self.calibration_matrix = mtx
            self.calibration_distance = dist
            
        
        return CameraCalibrationResult(self.calibration_matrix, self.calibration_distance, self.calibration_matrix)
        

In [6]:
class Viewport:    
    
    @staticmethod
    def y(image):
        height, width = image.shape[:2]
        viewport_y = math.floor(height / 2) + 50

        if height != 720:
            viewport_y += 50
        else:
            viewport_y += 100
        
        return viewport_y
    
    @staticmethod
    def whole_area_polygon(image):
        height, width = image.shape[:2]
        mid_of_viewport_x = math.floor(width / 2)
        viewport_y = math.floor(height / 2) #+ 50 + 90
        
        if height != 720:
            viewport_y += 50
        else:
            viewport_y += 100
        
        viewport_right_x = mid_of_viewport_x + 90
        viewport_left_x = mid_of_viewport_x - 90
        return Polygon.create_from_array(np.array([(viewport_left_x, viewport_y), (0, height), (width, height), (viewport_right_x, viewport_y)]))         
        
    @staticmethod
    def whole_area_to_perspective_transform_source(image):
        viewport_arr = Viewport.whole_area_polygon(image.image_data).arr
        src = np.float32([viewport_arr[0],
                          viewport_arr[3],
                          viewport_arr[1],
                          viewport_arr[2]])
        
        return src
    
    @staticmethod
    def whole_area_to_perspective_transform_target(image):
        height, width = image.image_data.shape[:2]
        dst = np.float32([(450,0),
                          (width-450,0),
                          (450,height),
                          (width-450,height)])
        
        return dst
        
        
    
    @staticmethod
    def right_area_polygon(image):
        height, width = image.shape[:2]
        mid_of_viewport_x = math.floor(width / 2)
        viewport_y = math.floor(height / 2) #+ 50 + 90

        if height != 720:
            viewport_y += 50
        else:
            viewport_y += 100
        
        viewport_right_x = mid_of_viewport_x + 30
        return Polygon.create_from_array(np.array([(mid_of_viewport_x + 1, viewport_y), (mid_of_viewport_x + 1, height), (width, height), (viewport_right_x, viewport_y)]))         
    
    @staticmethod
    def left_area_polygon(image):
        height, width = image.shape[:2]
        mid_of_viewport_x = math.floor(width / 2)
        viewport_y = math.floor(height / 2) #+ 50 + 90
        
        if height != 720:
            viewport_y += 50
        else:
            viewport_y += 100
        
        viewport_left_x = mid_of_viewport_x - 30
        return Polygon.create_from_array(np.array([(viewport_left_x, viewport_y), (0,height), (mid_of_viewport_x, height), (mid_of_viewport_x, viewport_y)]))         
    
    @staticmethod
    def get_mask_image(target_image):
        viewport_image = CvImage.create_black_image(target_image.width(), target_image.height(), target_image.channel_count())

        vertices = Viewport.whole_area_polygon(target_image.image_data)
        PolygonPlotter.draw_to(viewport_image, vertices.arr)
        
        return viewport_image
        
    
class Polygon:
    @staticmethod
    def create_from_array(arr):
        poly = Polygon(arr)
        
        return poly
        
    def __init__(self, arr):
        self.arr = arr
        
    def is_point_inside(self, point_tuple):
        result = cv2.pointPolygonTest(self.arr, point_tuple, False)         
        
        return (result >= 0)
        
    
class Line:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        
    @staticmethod
    def create_from_array(arr):
        x1, y1, x2, y2 = arr[0]
        new_line = Line(x1, y1, x2, y2)
        
        return new_line
    
    
    def abs_dx(self):
        return abs(self.dx())
        
    def abs_dy(self):
        return abs(self.dy())

    def dx(self):
        return self.x1 - self.x2
        
    def dy(self):
        return self.y1 - self.y2
    
    def slope(self):
        return self.dy() / self.dx()
    
    def is_inside(self, polygon):
        return (polygon.is_point_inside((self.x1, self.y1)) and polygon.is_point_inside((self.x2, self.y2)))
        
    
class LinePlotter:
    @staticmethod
    def draw_to(target_image, point1, point2, thickness = 4, bgr_color = (100, 255, 0)):
        cv2.line(target_image.image_data, point1, point2, bgr_color, thickness)
        
class CirclePlotter:
    @staticmethod
    def draw_to(target_image, center_point, radius, bgr_color = (100, 255, 0), solid_color = -1):
        cv2.circle(target_image.image_data, center_point, radius, bgr_color, solid_color)
        
class RectanglePlotter:
    @staticmethod
    def draw_to(target_image, left_point, right_point, bgr_color, solid_color = -2):
        cv2.rectangle(target_image.image_data, left_point, right_point, bgr_color, solid_color)
    
class PolygonPlotter:
    @staticmethod
    def draw_to(target_image, vertices, color_value = (255, 255, 255)):
        adapted_vertices = np.array([vertices], dtype=np.int32)
        cv2.fillPoly(target_image.image_data, adapted_vertices, color_value)
    

In [7]:
cvimage = CvImage.load_from_file('../test_images/calibration_test.png')

ImageViewer.display_image(cvimage.draw_chessboard_corners(8, 6))


In [301]:
calibration_image_files = File.list_folder('../camera_cal/')

calibration_image_files = list(map(lambda x: '../camera_cal/' + x, calibration_image_files))
# calibration_image_files.append('../test_images/calibration_test.png')

corners_hash = {0: (7, 6), 1: (7, 4), 2: (9, 6), 3: (9, 6), 4: (9, 6), 5: (9, 6), 6: (9, 5), 7: (9, 6), 8: (9, 6), 9: (9, 6), 10: (9, 6), 11: (9, 6), 12: (9, 6), 13: (9, 6), 14: (9, 6), 15: (9, 6), 16: (9, 6), 17: (9, 6), 18: (9, 6), 19: (9, 6), 20:(8,6)}

# index = 4
# cvimage = CvImage.load_from_file(calibration_image_files[index])

# ImageViewer.display_image(cvimage.draw_chessboard_corners(corners_hash[index][0], corners_hash[index][1]))


camera = Camera()

calibration_result = camera.calibrate(calibration_image_files, 9, 6)

print(calibration_result)



Not found for index:0
Not found for index:1
Not found for index:6
CameraCalibrationResult(calibration_matrix=array([[1.15693957e+03, 0.00000000e+00, 6.65948026e+02],
       [0.00000000e+00, 1.15213792e+03, 3.88785776e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), calibration_distance=array([[-2.37636474e-01, -8.54104018e-02, -7.90992506e-04,
        -1.15920533e-04,  1.05737433e-01]]), new_camera_matrix=array([[1.15693957e+03, 0.00000000e+00, 6.65948026e+02],
       [0.00000000e+00, 1.15213792e+03, 3.88785776e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]))


In [302]:
calibration_matrix=np.array([[1.15693957e+03, 0.00000000e+00, 6.65948026e+02],
       [0.00000000e+00, 1.15213792e+03, 3.88785776e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
calibration_distance=np.array([[-2.37636474e-01, -8.54104018e-02, -7.90992506e-04,
        -1.15920533e-04,  1.05737433e-01]])

calibration_result = CameraCalibrationResult(calibration_matrix, calibration_distance, calibration_matrix)

In [304]:
distorted_image = CvImage.load_from_file('../camera_cal/calibration2.jpg')

undistorted_image = distorted_image.undistort(calibration_result)

ImageViewer.display_image(undistorted_image)
# ImageViewer.display_image(distorted_image.draw_chessboard_corners(8,6))

## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [341]:
# Test undistortion on an image
img = cv2.imread('../camera_cal/calibration1.jpg')
img = cv2.imread('../test_images/calibration_test.png')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
# dst = cv2.undistort(img, mtx, dist, None, mtx)
dst = cv2.undistort(img, calibration_result.calibration_matrix, calibration_result.calibration_distance, None, calibration_result.calibration_matrix)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
# pickle.dump( dist_pickle, open( "calibration.p", "wb" ) )
#dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
f.subplots_adjust(hspace = .2, wspace=.05)
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)
print('...')

...


In [387]:
img = cv2.imread('../test_images/test2.jpg')


# viewport_arr = Viewport.whole_area_polygon(img).arr

src = Viewport.whole_area_to_perspective_transform_source(img)
dst = Viewport.whole_area_to_perspective_transform_target(img)

exampleImg_unwarp = cvimage.load_from_image_data(img).unwarp(src, dst)

ImageViewer.display_image(exampleImg_unwarp)


In [443]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2

# Load our image
binary_warped = mpimg.imread('../test_images/s1.png')

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    
    print(leftx_base, rightx_base)
    
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)

    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
#         # Identify the nonzero pixels in x and y within the window #
#         good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
#         (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
#         good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
#         (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
#         # Append these indices to the lists
#         left_lane_inds.append(good_left_inds)
#         right_lane_inds.append(good_right_inds)
        
#         # If you found > minpix pixels, recenter next window on their mean position
#         if len(good_left_inds) > minpix:
#             leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
#         if len(good_right_inds) > minpix:        
#             rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
#     try:
#         left_lane_inds = np.concatenate(left_lane_inds)
#         right_lane_inds = np.concatenate(right_lane_inds)
#     except ValueError:
#         # Avoids an error if the above is not implemented fully
#         pass

#     # Extract left and right line pixel positions
#     leftx = nonzerox[left_lane_inds]
#     lefty = nonzeroy[left_lane_inds] 
#     rightx = nonzerox[right_lane_inds]
#     righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

#     # Fit a second order polynomial to each using `np.polyfit`
#     left_fit = np.polyfit(lefty, leftx, 2)
#     right_fit = np.polyfit(righty, rightx, 2)

#     # Generate x and y values for plotting
#     ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
#     try:
#         left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
#         right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
#     except TypeError:
#         # Avoids an error if `left` and `right_fit` are still none or incorrect
#         print('The function failed to fit a line!')
#         left_fitx = 1*ploty**2 + 1*ploty
#         right_fitx = 1*ploty**2 + 1*ploty

#     ## Visualization ##
#     # Colors in the left and right lane regions
#     out_img[lefty, leftx] = [255, 0, 0]
#     out_img[righty, rightx] = [0, 0, 255]

#     # Plots the left and right polynomials on the lane lines
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')

    return out_img


out_img = fit_polynomial(binary_warped)

plt.imshow(out_img)

3 1683


error: OpenCV(4.5.1) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-yaf6rry6/opencv/modules/core/src/array.cpp:3229: error: (-215:Assertion failed) cn <= 4 in function 'scalarToRawData'


In [407]:
img = cv2.imread('../test_images/s1.png')

def hist(img):
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = img[img.shape[0]//2:,:]

    # Sum across image pixels vertically - make sure to set an `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half, axis=0)
    
    return histogram

histogram = hist(img)

# Visualize the resulting histogram
plt.plot(histogram)

In [402]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

calibration_matrix=np.array([[1.15693957e+03, 0.00000000e+00, 6.65948026e+02],
       [0.00000000e+00, 1.15213792e+03, 3.88785776e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
calibration_distance=np.array([[-2.37636474e-01, -8.54104018e-02, -7.90992506e-04,
        -1.15920533e-04,  1.05737433e-01]])

calibration_result = CameraCalibrationResult(calibration_matrix, calibration_distance, calibration_matrix)

def process_image(image):
    distorted_image = CvImage.load_from_image_data(image)
    undistorted_image = distorted_image.undistort(calibration_result)
        

    yellow_mask = undistorted_image.to_HSL().filter_inrange(np.array([0, 0, 75]),np.array([255, 255, 255]))
    white_mask = undistorted_image.to_HSL().filter_inrange(np.array([0, 211, 0]),np.array([255, 255, 255]))


    dark_image = yellow_mask.mask_or(white_mask).to_RGB().mask_and(Viewport.get_mask_image(undistorted_image))

    src = Viewport.whole_area_to_perspective_transform_source(dark_image)
    dst = Viewport.whole_area_to_perspective_transform_target(dark_image)

    exampleImg_unwarp = dark_image.unwarp(src, dst)
    
    
    return dark_image.image_data
    
    

In [403]:
white_output = '../test_videos_output/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("../project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Moviepy - Building video ../test_videos_output/project_video.mp4.
Moviepy - Writing video ../test_videos_output/project_video.mp4



Moviepy - Done !
Moviepy - video ready ../test_videos_output/project_video.mp4
CPU times: user 47.1 s, sys: 6.07 s, total: 53.1 s
Wall time: 24.6 s


In [404]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))